# Conditional Chains
- A Conditional Chain is like an "if-else" decision maker for AI workflows.
Instead of always running the same steps, the chain chooses the next step based on conditions.

Example in real life:
Imagine you are handling customer feedback:

- If the feedback is positive, send a thank-you reply.
- If the feedback is negative, send an apology and escalate to support.
- If the feedback is neutral, just acknowledge politely.

Here, the AI first classifies the sentiment, and then the chain decides which path to follow.

### In LangChain:

- Input comes in (e.g., customer review).
- The chain checks a condition (positive? negative? neutral?).
- Based on the result, it runs a specific chain of actions.
- If no condition matches, a default chain is used.



In [2]:
! pip install langchain-google-genai

In [48]:
# Importing Necessary Libraries
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableBranch
from google.colab import userdata


In [4]:
# Getting the API KEY
API_KEY = userdata.get('GOOGLE_API_KEY')


In [5]:
# Choosing the model
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash",
    google_api_key = API_KEY
)


In [54]:
# Providing 4 prompt Templates
positive_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human","Generate a thank you note for this positive feedback: {feedback}."),
    ]
)

negative_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human","Generate a response addressing this negative feedback: {feedback}."),
    ]
)

neutral_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human","Generate a request for more details for this neutral feedback: {feedback}."),
    ]
)

escalate_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "Generate a message to escalate this feedback to a human agent: {feedback}."),
    ]
)


In [53]:
# # Creating a prompt template for classification
classification_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Classify the sentiment of this feedback as positive, negative, neutral, or escalate: {feedback}."),
    ]
)


In [55]:
# Defining branches (different paths depending on sentiment)
# If output contains "positive", run the positive feedback chain, same for negative & neutral.
branches = RunnableBranch(
    (lambda x: "positive" in x, positive_feedback_template | llm | StrOutputParser()),
    (lambda x: "negative" in x, negative_feedback_template | llm | StrOutputParser()),
    (lambda x: "neutral" in x, neutral_feedback_template | llm | StrOutputParser()),
    escalate_feedback_template | llm | StrOutputParser()
)

# Default branch: if nothing matches, escalate the feedback
classification_chain = classification_template | llm | StrOutputParser()

In [56]:
# Main chain: takes input → classifies sentiment → returns result
chain = classification_chain | branches


In [58]:
# Providing review
review = "The product is terrible. It broke after just one use and the quality is very poor."

# Filling the chain with actual values
response = chain.invoke({
    "feedback": review
})

# Getting response
print(response)


Okay, here are a few options for escalating the feedback "Negative" to a human agent, depending on the context and your desired level of urgency:

**Option 1 (Simple and Direct):**

> Escalating due to negative feedback. Requires human review.

**Option 2 (Slightly More Specific):**

> Received negative feedback ("Negative"). Escalating for further investigation and potential action.

**Option 3 (If you have a specific escalation protocol):**

> Negative feedback received. Following protocol [Protocol Number/Name] for escalation.

**Option 4 (If you suspect a serious issue):**

> Urgent escalation. Received negative feedback ("Negative") which may indicate a significant problem. Requires immediate human attention.

**Option 5 (If you can provide a little more context):**

> Received negative feedback ("Negative") related to [briefly mention the topic, e.g., order fulfillment, product quality, customer service]. Escalating to a human agent for review.

**Key Considerations:**

*   **You